In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install sentence_transformers
!pip install git+https://github.com/pmbaumgartner/setfit


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pmbaumgartner/setfit to /tmp/pip-req-build-gmwur_4g
  Running command git clone -q https://github.com/pmbaumgartner/setfit /tmp/pip-req-build-gmwur_4g
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [3]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets, evaluation
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

import torch
import random

import json
from pandas import Series, DataFrame
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import gzip

from transformers import pipeline
import sqlite3 as db
from setfit import SetFitClassifier
from random import choice, randint, shuffle


from pathlib import Path
data_dir_pref = Path('/content/drive/MyDrive/WORK/data') 



def test_model_saved(name_model_saved, texts):
  clf = clf_test = SetFitClassifier.load(name_model_saved)
  selected = clf.predict(texts)
  return selected



tqdm.pandas()


In [4]:
""" Prepare dataset for train: 20 samples """

def read_db_to_df():
  """ Чтение размеченных текстов из БД в датафрейм """
  conn = db.connect('/content/drive/MyDrive/WORK/data/labels.db')
  train_df = pd.read_sql_query("SELECT text,label FROM labels", conn)
  train_df.sort_values("label", ascending=False).head()
  return train_df


In [5]:
data_df = read_db_to_df()
index = data_df.sort_values("label", ascending=False).index.to_list()
dataset = list(zip(data_df.text[index].to_list()[:20], data_df.label[index].to_list()))
shuffle(dataset)
docs = [i[0] for i in dataset] 
labels = [i[1] for i in dataset]

In [6]:
""" train model """

clf = SetFitClassifier('cross-encoder/quora-distilroberta-base')  #'sentence-transformers/all-mpnet-base-v1')   
clf.fit(docs, labels) 

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/607 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/cross-encoder_quora-distilroberta-base were not used when initializing RobertaModel: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/cross-encoder_quora-distilroberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probab

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [7]:
""" Prepare dataset for eval of new model """

shuffle(dataset)
docs_eval = [i[0] for i in dataset] 
labels_eval = [i[1] for i in dataset]


In [8]:
""" eval of new model """

y_pred_eval = clf.predict(docs_eval)

print(labels_eval)
print(y_pred_eval)
print('accuracy=', accuracy_score(labels_eval, y_pred_eval))

[0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
[0 1 0 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0]
accuracy= 0.95


In [9]:
""" save set-fit model """

path_model = '/content/drive/MyDrive/WORK/models/setfit_20.mdl'
clf.save(path_model)


**ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ FT МОДЕЛИ **

In [10]:
""" prepare data for using saved set-fit model"""

path_model = '/content/drive/MyDrive/WORK/models/setfit_20.mdl'


In [11]:
""" Подготовка набора из ппоследних 100 текстов комментариев """

name_texts_file = Path.joinpath(data_dir_pref, 'all_comments_texts.txt')
with open(name_texts_file, 'r') as file:
   texts = file.readlines()[:100]


In [12]:
""" using saved set-fit model"""

index_texts_selected = test_model_saved(path_model, texts)
print(sum(index_texts_selected))


23


In [13]:
""" get selected texts """


mask = np.array(index_texts_selected, dtype=bool)
text_selected = list(np.array(texts)[mask])


In [14]:
text_selected

["For another counterargument to your point about the fact that some positions don't look attractive to people who are overqualified, here's Ben West's article. I personally think that making the position challenge and a growth opportunity make people more motivated and excited.\n",
 "I'm now noticing I'm unsure whether you disagree that it's okay to approximate #1 as zero. I read your post as arguing against approximating #2 as zero, but maybe that's just because that's something I agree with, and you actually intended to cover both 1+2.\n",
 'FWIW, I wouldn\'t say I\'m "dumb," but I dropped out of a University of Minnesota counseling psychology undergrad degree and have spent my entire "EA" career (at MIRI then Open Phil) working with people who are mostly very likely smarter than I am, and definitely better-credentialed. And I see plenty of posts on EA-related forums that require background knowledge or quantitative ability that I don\'t have, and I mostly just skip those. Sometimes

In [16]:
""" save results - selected texts to file """


name_file_tests_selected = Path.joinpath(data_dir_pref, 'texts_model_selected_from_0_100_comments.txt')

with open(name_file_tests_selected, 'w') as file:
  file.writelines([r+'\n\n' for r in text_selected])

